## Here is the code to get the paper citations

In [1]:
import bs4 as bs
import urllib.request
import re as re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
#     def get_full_text(self):
#         self.html = urllib.request.urlopen(self.paper_url)# grab html
#         self.soup = bs.BeautifulSoup(self.html,'lxml')# trun pretty 
#         ## extract
#         self.citations = self.soup.find_all('div', attrs={'class':'References'})
#         self.cite_flag = True;
#         if self.citations is None:
#             print('cant get citations for ',self.paper_url) 
#         elif not self.citations: # NN
#             self.citations = self.soup.find('ol', {'class':'references'}).select('li[id]') # get citations
#             if self.citations is None:
#                 print('cant get citations for ',self.paper_url) 
#             elif not a.citations:
#                 print('cant get citations for ',self.paper_url)

In [ ]:

links_output = []
for link in a.soup.find_all('a'):
    links_output.append(link.get('href'))
    
for i in range(0,len(links_output)): 
    if links_output[i] is None:
        continue
    else:
        if '.full' in links_output[i]:
            if 'http:' in links_output[i]:
                master_link = links_output[i]
                break

In [88]:
# need to get j neuroscience to work better
class nerdy:
    def __init__(self, paper_url):
        if 'http://dx' in paper_url:
            print('a')
            self.paper_url = paper_url
        elif: 'http' in paper_url:
            print('b')
            self.paper_url = paper_url
        else:
            print('c')
            self.paper_url = 'http://dx.doi.org/' + paper_url

        
    def elsevier_ref(self):
        prefix = 'https://api.elsevier.com/content/abstract/doi/'
        suffix = '?apiKey=40acd4c83833cb8ead38b50834823fd9&view=REF'
        doi = self.paper_url

        self.link = prefix+doi+suffix
        self.html = urllib.request.urlopen(self.link)# grab html
        self.soup = bs.BeautifulSoup(self.html,'xml')# trun pretty 
        
        self.citations =[]
        for doi in self.soup.find_all('doi'):
            self.citations.append(doi.contents[0])
        
        self.random_ten = np.random.choice(self.citations,size = 10)
                        
    def get_full_text(self):
        # added error handeling
        req = urllib.request.Request(self.paper_url)
        try: urllib.request.urlopen(req)
        except urllib.error.URLError as e:
            print('link ' + e.reason)
            return
        else:
            self.html = urllib.request.urlopen(self.paper_url)
        
#         self.html = urllib.request.urlopen(self.paper_url)# grab html
        self.soup = bs.BeautifulSoup(self.html,'lxml')# trun pretty 
        ## extract
        ## auto detect journal
        if "Nature" in self.soup.title.get_text():
            print('Nature Journal')
            self.journal = 'Nature'
            self.citations = self.soup.find('ol', {'class':'references'}).select('li[id]')
            
            counter = 0
            prefix = ''#'http://dx.doi.org/'
            links_output = []
            for i in range(len(self.citations[0:])):
                if self.citations[i].a is None: # if link doesnt exist 
                    counter = counter + 1
                    print(i, 'cant find link')
                elif len(self.citations[i].a['href']) > 90: # if the link is really long 
                    print(i, 'long link')
#                     links_output.append(re.sub("http://dx.doi.org/", '' ,self.citations[i].a['href']))
                elif 'http://dx.doi.org/' in self.citations[i].a['href']: # normal
                    links_output.append(re.sub("http://dx.doi.org/", '' ,self.citations[i].a['href']))
                else: # if its short add prefix
                    links_output.append(re.sub("/doifinder/", '' ,self.citations[i].a['href']))
                    
            ## save em to class
            self.links_output = links_output
            self.missing_links = counter
            self.random_ten = np.random.choice(self.links_output,size = 10)
            
        elif "ScienceDirect" in self.soup.title.get_text(): 
            print(self.soup.title.get_text())
        elif "Science" in self.soup.title.get_text(): # science
            print(self.soup.title.get_text())
            
            print('getting full text','Science Journal')
            self.journal = 'Science'
            self.soup = bs.BeautifulSoup(self.html,'lxml')#        
            self.citations = self.soup.find_all('div', attrs={'class':'References'})
            if self.citations is None:
                print('actually abstract getting really')
                links_output = []
                for link in self.soup.find_all('a'):
                    links_output.append(link.get('href'))

                for i in range(0,len(links_output)): 
                    if links_output[i] is None:
                        continue
                    else:
                        if '.full' in links_output[i]:
                            if 'http:' in links_output[i]:
                                self.master_link = links_output[i]
                                break
                self.html = urllib.request.urlopen(self.master_link)# grab html
                self.soup = bs.BeautifulSoup(self.html,'lxml')#        
                self.citations = self.soup.find_all('div', attrs={'class':'References'})
                self.random_ten = np.random.choice(self.citations,size = 10)
            else:
                print(self.citations)
                self.random_ten = np.random.choice(self.citations,size = 10)

        elif "Elsevier" in self.soup.title.get_text(): # if elsevier use api
            prefix = 'https://api.elsevier.com/content/abstract/doi/'
            suffix = '?apiKey=40acd4c83833cb8ead38b50834823fd9&view=REF'
            doi = self.paper_url

            self.link = prefix+doi+suffix
            self.html = urllib.request.urlopen(self.link)# grab html
            self.soup = bs.BeautifulSoup(self.html,'xml')# trun pretty 

            self.citations =[]
            for doi in self.soup.find_all('doi'):
                self.citations.append(doi.contents[0])

            self.random_ten = np.random.choice(self.citations,size = 10)
        
    def get_links(self):    
        ## get links
        counter = 0
        prefix = ''#'http://dx.doi.org/'
        links_output = [] # create output 
        if self.cite_flag == True:
            for i in range(len(self.citations[0:])):
                if len(self.citations[i].find_all('a')) == 4:
                    links_output.append(self.citations[i].find_all('a')[3]['href'])
                elif len(self.citations[i].find_all('a')) < 2 :
                    counter = counter + 1
                    print(i,'cant find link')
                elif 'http://dx.doi.org/' in self.citations[i].find_all('a')[1]['href']:
                    links_output.append(self.citations[i].find_all('a')[1]['href'])
                else:
                    counter = counter + 1
                    print(i,'cant find link')

        else: ## v1 
            for i in range(len(self.citations[0:])):
                if self.citations[i].a is None: # if link doesnt exist 
                    counter = counter + 1
                    print(i, 'cant find link')
                elif len(self.citations[i].a['href']) > 120: # if the link is really long 
                    links_output.append(re.sub("http://dx.doi.org/", '' ,self.citations[i].a['href']))
                elif 'http://dx.doi.org/' in self.citations[i].a['href']: # normal
                    links_output.append(self.citations[i].a['href'])
                else: # if its short add prefix
                    links_output.append(prefix + re.sub("/doifinder/", '' ,self.citations[i].a['href']))
                    
        ## save em to class
        self.links_output = links_output
        self.missing_links = counter
        self.random_ten = np.random.choice(self.links_output,size = 10)

SyntaxError: invalid syntax (<ipython-input-88-2ce3a1234360>, line 7)

In [15]:
a = nerdy('http://dx.doi.org/10.1038/35049064')
a.get_full_text()

Nature Journal
0 long link
2 long link
3 long link
5 long link
7 long link
8 long link
9 long link
11 long link
14 long link
15 cant find link
16 long link
17 long link
20 long link
21 long link
22 long link
26 long link
29 long link
30 long link
32 long link
33 long link
34 long link
35 long link
37 long link
39 cant find link
42 long link
44 long link
47 long link
49 long link
50 long link
51 cant find link
56 long link
57 long link
61 long link
62 long link
63 long link
64 long link
65 long link
66 long link
67 long link
71 long link
72 long link
73 long link
76 long link
77 long link
78 long link
79 cant find link
82 long link
84 long link
85 long link
86 long link
87 long link
88 long link
91 long link
93 long link
94 long link
96 long link
97 long link
101 long link
103 long link
104 long link
105 long link
109 cant find link
112 long link
114 long link
115 long link
122 cant find link
124 long link
127 long link
128 cant find link
130 long link
132 long link
134 cant find link
1

In [16]:
a.random_ten

array(['10.1006/nimg.1995.1038', '10.1016/S0896-6273(00)80315-3',
       '10.1002/(SICI)1096-9861(19970602)382:2<153::AID-CNE2>3.3.CO;2-#',
       '10.1126/science.8290960', '10.1037//0033-295X.87.3.252',
       '10.1016/S0301-0082(98)00002-1', '10.1037//0735-7044.114.3.459',
       '10.1037//0735-7044.111.3.467', '10.1126/science.8290960',
       '10.1037//0735-7044.108.1.11'], 
      dtype='<U63')

In [56]:
a=nerdy('10.1006/nimg.1995.1038')
a.get_full_text()
# html = urllib.request.urlopen(a.paper_url)

link Not Found


In [35]:
req = urllib.request.Request('http://dx.doi.org/10.1006/nimg.1995.1038')
try: urllib.request.urlopen(req)
except urllib.error.URLError as e:
    print('link ' + e.reason)

link Not Found


In [32]:
req = urllib.request.Request('http://dx.doi.org/10.1006/nimg.1995.1038')

In [31]:
req

In [27]:
urllib.request.HTTPError('http://dx.doi.org/10.1006`/nimg.1995.1038')

TypeError: __init__() missing 4 required positional arguments: 'code', 'msg', 'hdrs', and 'fp'

In [23]:
urllib.error.HTTPError('http://dx.doi.org/10.1006/nimg.1995.1038')

TypeError: __init__() missing 4 required positional arguments: 'code', 'msg', 'hdrs', and 'fp'

In [204]:
a.random_ten[1]

'10.1016/0166-4328(96)00006-X'

In [173]:
a.citations[1]

<li id="B2"><p class="details">Gaffan, D. &amp; 
Murray, E. A. <span class="atl">Monkeys (<i>Macaca fascicularis
</i>) with rhinal cortex ablations succeed in object discrimination learning
despite 24-hr intertrial intervals and fail at matching to sample despite
double sample presentations</span>. <span class="jtl">Behav. Neurosci.</span> <span class="vid">106</span>, 
30–38 (<span class="cite-month-year">1992</span>).</p><ul class="article-views"><li class="ref-doi first"><a href="http://dx.doi.org/10.1037//0735-7044.106.1.30" title="">Article</a></li><li class="ref-medline"><a href="http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?holding=npg&amp;cmd=Retrieve&amp;db=PubMed&amp;list_uids=1554436&amp;dopt=Abstract" title="Article on PubMed - ">PubMed</a></li><li class="ref-isi"><a href="http://links.isiglobalnet2.com/gateway/Gateway.cgi?&amp;GWVersion=2&amp;SrcAuth=Nature&amp;SrcApp=Nature&amp;DestLinkType=FullRecord&amp;KeyUT=A1992HF35800003&amp;DestApp=WOS_CPL" title="Article on ISI - 

In [176]:
for link in a.citations:
    print(link.find_all('a').select('href'))

AttributeError: 'ResultSet' object has no attribute 'select'

In [164]:
a = nerdy("http://science.sciencemag.org/content/285/5433/1582")
a.get_full_text()

getting full text Science Journal


In [165]:
a.citations

[]

In [143]:
for link in a.soup.find_all()
# a.soup.find_all('div',{'class':'section ref-list' })

[<div class="section ref-list" id="ref-list-1"><h2 class="">REFERENCES AND NOTES</h2><ol class="cit-list"><li><a class="rev-xref-ref" href="#xref-ref-1-1" id="ref-1" title="View reference 1 in text">↵</a><div class="cit ref-cit ref-journal" data-doi="10.1126/science.1896849" id="cit-285.5433.1582.1"><div class="cit-metadata"><ol class="cit-auth-list"><li><span class="cit-auth"> <span class="cit-name-given-names">L. R.</span> <span class="cit-name-surname">Squire</span></span>, </li><li><span class="cit-auth"> <span class="cit-name-given-names">S.</span> <span class="cit-name-surname">Zola-Morgan</span></span></li></ol><cite>, <abbr class="cit-jnl-abbrev">Science</abbr> <span class="cit-vol">253</span>, <span class="cit-fpage">1380</span> (<span class="cit-pub-date">1991</span>).</cite></div><div class="cit-extra"><a class="cit-ref-sprinkles cit-ref-sprinkles-openurl cit-ref-sprinkles-open-url" href="{openurl}?query=rft.jtitle%253DScience%26rft.stitle%253DScience%26rft.aulast%253DSquire

In [119]:
links_output = []
for link in a.soup.find_all('a'):
    links_output.append(link.get('href'))
    
for i in range(0,len(links_output)): 
    if links_output[i] is None:
        continue
    else:
        if '.full' in links_output[i]:
            if 'http:' in links_output[i]:
                master_link = links_output[i]
                break


In [120]:
master_link

'http://science.sciencemag.org/content/285/5433/1582.full'

In [67]:
a.soup.find_all('a').attrs('href')

AttributeError: 'ResultSet' object has no attribute 'attrs'

In [158]:
a.citations[15].p

<p class="details">Suzuki, W. A. &amp; Amaral, D. G.                    <span class="atl">Perirhinal and parahippocampal cortices of the macaque monkey: cortical afferents</span>. <span class="jtl">J. Comp. Neurol.</span> <span class="vid">350</span>, 497–533 (<span class="cite-month-year">1994</span>).</p>

In [153]:
for i in range(0,len(a.citations)):
    if a.citations[i].a is None:
        continue
        print(i,'nolink')
    print(i, a.citations[i].a['href'])

0 http://dx.doi.org/10.1126/science.1896849
1 /doifinder/10.1038/35049064
2 http://dx.doi.org/10.1146/annurev.neuro.30.051606.094328
3 http://dx.doi.org/10.1098/rstb.1997.0132
4 /doifinder/10.1038/nrn2733
5 http://dx.doi.org/10.1093/brain/110.6.1631
6 http://dx.doi.org/10.1002/1098-1063(2000)10:4<398::AID-HIPO6>3.0.CO;2-K
7 http://dx.doi.org/10.1016/j.neubiorev.2011.09.005
8 http://dx.doi.org/10.1002/hipo.20314
9 http://dx.doi.org/10.1002/hipo.22024
10 http://dx.doi.org/10.1002/(SICI)1096-9861(19961125)375:4<552::AID-CNE2>3.0.CO;2-0
11 http://dx.doi.org/10.1152/jn.00077.2008
12 http://dx.doi.org/10.1523/JNEUROSCI.3711-11.2012
13 http://dx.doi.org/10.1002/cne.10883
14 http://dx.doi.org/10.1002/cne.21346
15 http://dx.doi.org/10.1002/cne.903500402
16 http://dx.doi.org/10.1002/cne.20079
17 http://dx.doi.org/10.1002/cne.10243
18 http://dx.doi.org/10.1002/cne.20796
19 /doifinder/10.1038/nrn3008
20 http://dx.doi.org/10.1002/cne.902250105
21 http://dx.doi.org/10.1073/pnas.98.2.676
22 http://dx

In [17]:
"Nature" in a.soup.title.get_text()

True

In [146]:
a = nerdy('http://www.nature.com/nrn/journal/v13/n10/full/nrn3338.html')
a.get_full_text()

Nature Journal


In [152]:
a.citations[1].a['href']

'/doifinder/10.1038/35049064'

In [ ]:
html=urllib.request.urlopen('http://dx.doi.org/10.1038/35049064')
soup = bs.BeautifulSoup(html,'lxml')# trun pretty 
## extract
citations = soup.find_all('ol', attrs={'class':'references'})

In [ ]:
a = nerdy('http://www.nature.com/nrn/journal/v2/n1/full/nrn0101_051a.html')
a.get_full_text()
a.citations
a.citations[1].a['href']

In [ ]:
# a = nerdy('http://www.nature.com/nrn/journal/v13/n10/full/nrn3338.html')
a = nerdy('http://www.nature.com/nrn/journal/v13/n10/full/nrn3338.html') # nature neuro
a.get_full_text()
a.get_links()


In [ ]:
a = nerdy('http://www.jneurosci.org/content/32/19/6550')
a.get_full_text()

In [ ]:
a.soup

In [ ]:
a.soup

In [ ]:
a.soup.find_all('div', {'class':"cit ref-cit ref-journal"})[0]

In [ ]:
a.soup.find_all('div', {'class':"cit ref-cit ref-journal"})[

In [ ]:
a = 0
a = nerdy('http://www.sciencedirect.com/science/article/pii/S0010027716301998')# doesnt work # elsivier 

In [ ]:
a = nerdy('http://journal.frontiersin.org/article/10.3389/fnhum.2014.00073/full') # frontiers works
a.get_full_text()
a.get_links()

In [ ]:
a.citations[18]

In [ ]:
a.citations[1].find_all('a')[1]['href']

In [ ]:
a.links_output

In [ ]:
a.get_links()

In [ ]:
a.random_ten

In [44]:
import networkx as nx

In [ ]:
DG = nx.DiGraph()

def recursive_add(DOI)
    if not DG.has_node(DOI) :
        node_data = 

In [89]:
# this is where the magic happens
def recursive_add(DOI):
    if not DG.has_node( DOI ):
        node_data = nerdy(DOI)
        print(node_data.paper_url)
#         node_data.get_full_text()
#         node_data.get_links()
        node_data.get_full_text()

#         node_data.elsevier_ref()    
        DG.add_node( DOI)
        child_list = node_data.random_ten
#         return(child_list)
        print(DOI,child_list)
#     for child in child_list:
#         print(child)
#         recursive_add(child)
#         print( 'addding: '+child )
#         DG.add_edge(DOI, child)
    

In [90]:
DG = nx.DiGraph()
recursive_add('https://www.nature.com/neuro/journal/vaop/ncurrent/full/nn.4572.html')

http://dx.doi.org/https://www.nature.com/neuro/journal/vaop/ncurrent/full/nn.4572.html
link Not Found


AttributeError: 'nerdy' object has no attribute 'random_ten'

In [ ]:
b = nerdy(a.random_ten[2])

In [ ]:
urllib.request.urlopen(b.paper_url)

In [ ]:
b.get_full_text()

In [ ]:
b.

In [ ]:
b.soup

In [ ]:
# b.get_full_text()
b.soup

In [ ]:
recursive_add(a.random_ten[1])